In [1]:
import yaml
import os
import numpy as np
import json

In [2]:
input_folder = "./k8s_raw/"
output_folder= "./k8s_converted/"

In [3]:
qualities = [1,2,3]
instances = [1,2,3,4,5]


In [4]:
def change_quality_complexisty(config, quality):
    config_json = json.loads(config)
    config_json["endpoints"][0]["cpu_complexity"]["execution_time"] *=quality
    config_str = json.dumps(config_json)
    return config_str.replace("\n", "")

In [5]:
try:
    os.mkdir(output_folder)
except:
    os.system("rm -rf "+output_folder)
    os.mkdir(output_folder)

In [6]:
config_files = os.listdir(input_folder)

In [7]:
for file in config_files:

    for q in qualities:
        for i in instances:
                with open(input_folder+file, "r") as f:
                    config = list(yaml.safe_load_all(f))
                    service_name = file.replace(".yaml","")
                    svc_label = "-%d-%d" % (q, i)
                    # Update service name
                    config[0]["metadata"]["name"]+=svc_label
                    config[0]["metadata"]["labels"]["name"] +=svc_label
                    config[0]["data"]["conf.json"] = change_quality_complexisty(config[0]["data"]["conf.json"],q)

                    config[1]["metadata"]["name"]+=svc_label
                    config[1]["spec"]["selector"]["matchLabels"]["app"]+=svc_label
                    # set init replica to 0
                    config[1]["spec"]["replicas"] = 0
                    config[1]["spec"]["template"]["metadata"]["labels"]["app"] +=svc_label
                    # set node to edge node name
                    config[1]["spec"]["template"]["spec"]["nodeName"] = "dummy"
                    config[1]["spec"]["template"]["spec"]["containers"][0]["env"][0]["value"]+=svc_label
                    # Add cpu assignment
                    config[1]["spec"]["template"]["spec"]["containers"][0]["env"].append({'name':"CPU_REQUEST",'value':str(int(int(config[1]["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]["cpu"].replace("m",""))/100))})
                    # Set initial delay
                    config[1]["spec"]["template"]["spec"]["containers"][0]["readinessProbe"]["initialDelaySeconds"] = 0
                    config[1]["spec"]["template"]["spec"]["volumes"][0]["configMap"]["name"]+=svc_label

                    config[2]["metadata"]["name"]+=svc_label
                    config[2]["spec"]["selector"]["app"]+=svc_label

                    # output
                    output_file_name=service_name+svc_label+".yaml"
                    yaml.safe_dump_all(config)
                    with open(output_folder+output_file_name,"w") as f2:
                        yaml.safe_dump_all(config,f2,)


In [8]:
config[0]["data"]["conf.json"]

'{"processes": 2, "logging": true, "protocol": "http", "endpoints": [{"name": "endpoint1", "execution_mode": "parallel", "cpu_complexity": {"execution_time": 0.15000000000000002, "threads": 1}, "network_complexity": {"forward_requests": "asynchronous", "response_payload_size": 512, "called_services": null}}]}'